In [1]:
import numpy as np; np.random.seed(0)
import tensorflow as tf; tf.set_random_seed(seed=0)
import pandas as pd
import lightgbm as lgb
import gc
from collections import defaultdict
import matplotlib

In [2]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]
target = train.target.values

In [3]:
train_df = pd.read_pickle('data/nn_data_preprocessed_train.pkl')
test_df = pd.read_pickle('data/nn_data_preprocessed_test.pkl')
real_samples_indexes = pd.read_pickle('data/test_real_samples_indexes')

In [4]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing import text, sequence
import numpy as np
from keras.layers import Input, SpatialDropout1D,Dropout, GlobalAveragePooling1D, GlobalMaxPooling1D, \
                            CuDNNGRU, GRU, Bidirectional, CuDNNLSTM, Dense, Embedding, concatenate, Embedding, \
                            Flatten, Activation, BatchNormalization, regularizers, Conv1D, Conv2D, MaxPooling2D, Reshape, PReLU, \
                            GaussianNoise, Multiply, Add
from keras.constraints import max_norm
from keras.callbacks import *
from keras.initializers import Orthogonal
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LambdaCallback, Callback, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
import keras.backend as K
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import os
import pickle
import gc; gc.enable()
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import string
from scipy.stats import boxcox
import re
from sklearn.model_selection import StratifiedKFold, KFold
#from tqdm import tqdm

# check gpu
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13761802105025652882
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10729011200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18187896202318552121
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
# build model
from keras.layers import *
from keras.models import Model

from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, activations
from keras.metrics import *

def build_model():
    # share components
    inputs = Input(shape=(1, 2))
 
    main = Dense(64, activation='relu')(inputs)
    main = Dense(32, activation='relu')(main)
    main = Flatten()(main)

    outputs = Dense(1, activation = 'sigmoid')(main) # 1 class to be classified
    
    model = Model(inputs, outputs)
    model.regularizers = [regularizers.l2(0.0001)]
    
    # use over weights
    model.compile(optimizer = Adam(lr=0.001, clipnorm=1.), loss="binary_crossentropy")
    
    #model.summary()
    return model

model = build_model()
model.summary()
K.clear_session()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 2)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 64)             192       
_________________________________________________________________
dense_2 (Dense)              (None, 1, 32)             2080      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,305
Trainable params: 2,305
Non-trainable params: 0
_________________________________________________________________


In [6]:
# training
from datetime import datetime
from keras_tqdm import TQDMNotebookCallback
from ipywidgets import IntProgress
from sklearn.model_selection import KFold, StratifiedKFold
import warnings; warnings.filterwarnings('ignore') 
from sklearn.metrics import accuracy_score, roc_auc_score

class auc_score_monitor(Callback):
    def __init__(self, val_data, val_target, checkpoint_file, min_lr =1e-5, reduce_lr_patience=2, early_stop_patience=4, factor=0.1):
        self.val_data = val_data
        self.val_target = val_target
        self.checkpoint_file = checkpoint_file
        self.reduce_lr_patience = reduce_lr_patience
        self.early_stop_patience = early_stop_patience
        self.best_val_score = 0
        self.epoch_num = 0
        self.factor = factor
        self.unimproved_lr_counter = 0
        self.unimproved_stop_counter = 0
        self.min_lr = min_lr
        
    def on_train_begin(self, logs={}):
        self.val_scores = []
        
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict(self.val_data).reshape((-1,))
        val_score = roc_auc_score(self.val_target, val_pred)
        # clip pred
        self.val_scores.append(val_score)
        
        #print(self.val_target, '\n', val_pred)
        print('Epoch {} val_score: {:.5f}'.format(self.epoch_num, val_score))
        self.epoch_num += 1
        
        if val_score > self.best_val_score:
            print ('Val Score improve from {:5f} to {:5f}'.format(self.best_val_score, val_score))
            self.best_val_score = val_score
            self.unimproved_lr_counter = 0
            self.unimproved_stop_counter = 0
            if self.checkpoint_file is not None:
                print('Saving file to', self.checkpoint_file)
                self.model.save_weights(self.checkpoint_file)
        else:
            if val_score<self.best_val_score:
                print('no improve from {:.5f}'.format(self.best_val_score))
                self.unimproved_lr_counter += 1
                self.unimproved_stop_counter += 1
            
        if self.reduce_lr_patience is not None and self.unimproved_lr_counter >= self.reduce_lr_patience:
            current_lr = K.eval(self.model.optimizer.lr)
            if current_lr > self.min_lr:
                print('Reduce LR from {:.6f} to {:.6f}'.format(current_lr, current_lr*self.factor))
                K.set_value(self.model.optimizer.lr, current_lr*self.factor)
                #self.model.load_weights(self.checkpoint_file)
            else:
                pass
            
            self.unimproved_lr_counter = 0
            
        if self.early_stop_patience is not None and self.unimproved_stop_counter >= self.early_stop_patience:
            print('Early Stop Criteria Meet')
            self.model.stop_training = True
                
        return

In [7]:
import keras
import gc

def special_reshape(vals):
    return np.vstack([v.reshape((2,-1)).T.reshape((1, -1, 2)) for v in vals])

class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, positive_rate=1., negative_rate=1.,
                 pl_data=None, pl_soft_label=None, pl_sample_rate=1.):
        #'Initialization'
        self.batch_size = batch_size
        self.X = X
        self.y = y
        self.positive_rate = positive_rate
        self.negative_rate = negative_rate 
        self.pl_data = pl_data
        self.pl_soft_label = pl_soft_label
        self.pl_sample_rate = pl_sample_rate
        self.on_epoch_end()
        
    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.resampled_y) / self.batch_size))

    def __getitem__(self, index):
        #'Generate one batch of data'
        # Generate indexes of the batch
        start = index*self.batch_size
        end = min((index+1)*self.batch_size, len(self.resampled_y))
        indexes = np.arange(len(self.resampled_y))[start: end]

        # Generate data
        return self.resampled_X[indexes,:,:], self.resampled_y[indexes]

    def on_epoch_end(self):
        # resample + shuffle
        feat_len = self.X.shape[1]//2
 
        X_p = self.X[self.y==1]
        X_n = self.X[self.y==0]
        
        pos_size = int(self.positive_rate*X_p.shape[0])
        X_p_new = np.zeros((pos_size, X_p.shape[1])).astype(np.float32)
        neg_size = int(self.negative_rate*X_n.shape[0])
        X_n_new = np.zeros((neg_size, X_n.shape[1])).astype(np.float32)
        
        for f in range(feat_len):
            pos_idx = np.random.choice(np.arange(X_p.shape[0]), size=pos_size, replace=True)
            X_p_new[:, f] = X_p[pos_idx,f]
            X_p_new[:, f+feat_len] = X_p[pos_idx,f+feat_len]
            
            neg_idx = np.random.choice(np.arange(X_n.shape[0]), size=neg_size, replace=True)
            X_n_new[:, f] = X_n[neg_idx,f]
            X_n_new[:, f+feat_len] = X_n[neg_idx,f+feat_len]
            
        self.resampled_X = np.vstack([X_p_new, X_n_new])
        self.resampled_y = np.array([1]*pos_size+[0]*neg_size)
        
        seq = np.random.choice(np.arange(len(self.resampled_y)), size=len(self.resampled_y), replace=False)
        self.resampled_X = special_reshape(self.resampled_X[seq, :])
        self.resampled_y = self.resampled_y[seq]
        #print(self.resampled_X.shape, self.resampled_y.shape)

In [8]:
seed = 0
train_epochs = 50
batch_size=32 # 32 or 64 is good (too huge for my PC), 128 is worse in the past experiments
cpu_count=4  
n_classses = 1
fold_num = 4
model_prefix = 'nn-per-feat-aug-v1' #'rnn-with-marcus-features-v4'
bags = 10
pseudo_label = False
pseudo_label_sample_rate = 0.8

In [9]:
#from clr_callback import CyclicLR

for f in feature_cols:
    fold = 0  
    retrain_fold = None

    feats = [f, f+'_freq']
    for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
        fold += 1

        print("Feat = {}, fold = {}".format(f, fold))
        
        if retrain_fold is not None and fold not in retrain_fold:
            continue

        #if fold < 3: continue
        
        model = build_model()
        file_path = "model_weights/{}_fold_{}_feat_{}.hdf5".format(model_prefix, fold, f)

        '''
        tr = train_df[feats].values[tr_ix,:]
        tr_y = target[tr_ix]
        
        val = special_reshape(train_df[feats].values[val_ix,:])
        val_y = target[val_ix].reshape((-1,1))

        lrs = [0.001]*1+[0.0001]*2
        lr_schd = LearningRateScheduler(lambda ep: lrs[ep], verbose=1)
        wmlog_loss_monitor = auc_score_monitor(val, val_y, 
                                               checkpoint_file=None, reduce_lr_patience=None, early_stop_patience=None, 
                                               factor=None) # calculate weighted m log loss per epoch
        
        training_generator = DataGenerator(tr, tr_y, batch_size=batch_size, positive_rate=2., negative_rate=1.,
                                           pl_data=None, pl_soft_label=None, pl_sample_rate=None)
        history = model.fit_generator(generator=training_generator,
                                      validation_data=(val, val_y),
                                      use_multiprocessing=False,
                                      workers=1, 
                                      epochs=len(lrs),
                                      verbose = 0, 
                                      callbacks = [#early_stop, lr_schd, check_point,
                                                   lr_schd, wmlog_loss_monitor,
                                                   TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
        del training_generator; gc.collect()
        '''
        
        tr = special_reshape(train_df[feats].values[tr_ix,:])
        val = special_reshape(train_df[feats].values[val_ix,:])
        tr_y = target[tr_ix]
        val_y = target[val_ix]

        lrs = [0.001]*1+[0.0001]*2
        lr_schd = LearningRateScheduler(lambda ep: lrs[ep], verbose=1)
        wmlog_loss_monitor = auc_score_monitor(val, val_y, 
                                                    checkpoint_file=None, reduce_lr_patience=None, early_stop_patience=None, 
                                                    factor=None) # calculate weighted m log loss per epoch

        history = model.fit(x=tr, y=tr_y,
                            #validation_data=(val, val_y),
                            epochs=len(lrs), verbose = 0, batch_size=batch_size,
                            callbacks = [lr_schd,
                                         wmlog_loss_monitor, TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

        
        
        model.save_weights(file_path)
        K.clear_session()

Feat = var_0, fold = 1



Epoch 00001: LearningRateScheduler setting learning rate to 0.001.


Epoch 0 val_score: 0.54858
Val Score improve from 0.000000 to 0.548581

Epoch 00002: LearningRateScheduler setting learning rate to 0.0001.


Epoch 1 val_score: 0.54752
no improve from 0.54858

Epoch 00003: LearningRateScheduler setting learning rate to 0.0001.


Epoch 2 val_score: 0.54941
Val Score improve from 0.548581 to 0.549406

Feat = var_0, fold = 2


...

Epoch 2 val_score: 0.52389
no improve from 0.52572

Feat = var_199, fold = 4



Epoch 00001: LearningRateScheduler setting learning rate to 0.001.


Epoch 0 val_score: 0.51718
Val Score improve from 0.000000 to 0.517184

Epoch 00002: LearningRateScheduler setting learning rate to 0.0001.


Epoch 1 val_score: 0.51924
Val Score improve from 0.517184 to 0.519239

Epoch 00003: LearningRateScheduler setting learning rate to 0.0001.


Epoch 2 val_score: 0.51931
Val Score improve from 0.519239 to 0.519310



In [12]:
#from clr_callback import CyclicLR

model = build_model()
train_oof = np.zeros((train_df.shape[0], len(feature_cols)), dtype=np.float32)
test_oof = np.zeros((test_df.shape[0], len(feature_cols)), dtype=np.float32)

for f_i, f in enumerate(feature_cols):
    fold = 0  
    
    feats = [f, f+'_freq']
    for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
        fold += 1
        
        file_path = "model_weights/{}_fold_{}_feat_{}.hdf5".format(model_prefix, fold, f)
        model.load_weights(file_path)
        
        val = special_reshape(train_df[feats].values[val_ix,:])
        val_y = target[val_ix]
        val_pred = model.predict(val).reshape((len(val_ix),))
        tst_pred = model.predict(special_reshape(test_df[feats].values)).reshape((test_df.shape[0],))
        
        train_oof[val_ix, f_i] = val_pred
        test_oof[:, f_i] += tst_pred/fold_num
        
        print("Feat = {}, fold = {}, roc_auc = {:.4f}".format(f, fold, roc_auc_score(val_y, val_pred)))
        
    print('{} overall auc = {:.4f}'.format(f, roc_auc_score(target, train_oof[:, f_i])))
    
K.clear_session()

Feat = var_0, fold = 1, roc_auc = 0.5494
Feat = var_0, fold = 2, roc_auc = 0.5442
Feat = var_0, fold = 3, roc_auc = 0.5477
Feat = var_0, fold = 4, roc_auc = 0.5479
var_0 overall auc = 0.5474
Feat = var_1, fold = 1, roc_auc = 0.5383
Feat = var_1, fold = 2, roc_auc = 0.5491
Feat = var_1, fold = 3, roc_auc = 0.5510
Feat = var_1, fold = 4, roc_auc = 0.5392
var_1 overall auc = 0.5437
Feat = var_2, fold = 1, roc_auc = 0.5510
Feat = var_2, fold = 2, roc_auc = 0.5532
Feat = var_2, fold = 3, roc_auc = 0.5448
Feat = var_2, fold = 4, roc_auc = 0.5499
var_2 overall auc = 0.5478
Feat = var_3, fold = 1, roc_auc = 0.5015
Feat = var_3, fold = 2, roc_auc = 0.5050
Feat = var_3, fold = 3, roc_auc = 0.5051
Feat = var_3, fold = 4, roc_auc = 0.5067
var_3 overall auc = 0.5033
Feat = var_4, fold = 1, roc_auc = 0.5126
Feat = var_4, fold = 2, roc_auc = 0.5038
Feat = var_4, fold = 3, roc_auc = 0.5138
Feat = var_4, fold = 4, roc_auc = 0.5033
var_4 overall auc = 0.5058
Feat = var_5, fold = 1, roc_auc = 0.5259
Feat

Feat = var_83, fold = 4, roc_auc = 0.5162
var_83 overall auc = 0.5186
Feat = var_84, fold = 1, roc_auc = 0.5020
Feat = var_84, fold = 2, roc_auc = 0.5094
Feat = var_84, fold = 3, roc_auc = 0.5113
Feat = var_84, fold = 4, roc_auc = 0.5135
var_84 overall auc = 0.5060
Feat = var_85, fold = 1, roc_auc = 0.5220
Feat = var_85, fold = 2, roc_auc = 0.5249
Feat = var_85, fold = 3, roc_auc = 0.5229
Feat = var_85, fold = 4, roc_auc = 0.5163
var_85 overall auc = 0.5207
Feat = var_86, fold = 1, roc_auc = 0.5371
Feat = var_86, fold = 2, roc_auc = 0.5406
Feat = var_86, fold = 3, roc_auc = 0.5431
Feat = var_86, fold = 4, roc_auc = 0.5387
var_86 overall auc = 0.5389
Feat = var_87, fold = 1, roc_auc = 0.5290
Feat = var_87, fold = 2, roc_auc = 0.5328
Feat = var_87, fold = 3, roc_auc = 0.5423
Feat = var_87, fold = 4, roc_auc = 0.5353
var_87 overall auc = 0.5322
Feat = var_88, fold = 1, roc_auc = 0.5157
Feat = var_88, fold = 2, roc_auc = 0.5147
Feat = var_88, fold = 3, roc_auc = 0.5176
Feat = var_88, fold 

Feat = var_165, fold = 2, roc_auc = 0.5554
Feat = var_165, fold = 3, roc_auc = 0.5512
Feat = var_165, fold = 4, roc_auc = 0.5471
var_165 overall auc = 0.5472
Feat = var_166, fold = 1, roc_auc = 0.5429
Feat = var_166, fold = 2, roc_auc = 0.5487
Feat = var_166, fold = 3, roc_auc = 0.5520
Feat = var_166, fold = 4, roc_auc = 0.5404
var_166 overall auc = 0.5456
Feat = var_167, fold = 1, roc_auc = 0.5297
Feat = var_167, fold = 2, roc_auc = 0.5305
Feat = var_167, fold = 3, roc_auc = 0.5259
Feat = var_167, fold = 4, roc_auc = 0.5295
var_167 overall auc = 0.5286
Feat = var_168, fold = 1, roc_auc = 0.5138
Feat = var_168, fold = 2, roc_auc = 0.5104
Feat = var_168, fold = 3, roc_auc = 0.5047
Feat = var_168, fold = 4, roc_auc = 0.5068
var_168 overall auc = 0.5073
Feat = var_169, fold = 1, roc_auc = 0.5416
Feat = var_169, fold = 2, roc_auc = 0.5377
Feat = var_169, fold = 3, roc_auc = 0.5423
Feat = var_169, fold = 4, roc_auc = 0.5353
var_169 overall auc = 0.5363
Feat = var_170, fold = 1, roc_auc = 0.

In [11]:
train_oof_mean = train_oof.mean(axis=1)

In [12]:
roc_auc_score(target, train_oof_mean)

0.8821497212601566

In [13]:
fold = 0
for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
    fold += 1

    val_y = target[val_ix]
    val_pred = train_oof_mean[val_ix]
    
    print("Fold = {}, roc_auc = {:.4f}".format(fold, roc_auc_score(val_y, val_pred)))

Fold = 1, roc_auc = 0.9030
Fold = 2, roc_auc = 0.9029
Fold = 3, roc_auc = 0.9080
Fold = 4, roc_auc = 0.9093


In [17]:
new_feats = [f+'_nn_oof' for f in feature_cols]
pd.DataFrame(data=train_oof, columns=new_feats).to_pickle('features/nn_per_feat_oof_raw_train')
pd.DataFrame(data=test_oof, columns=new_feats).to_pickle('features/nn_per_feat_oof_raw_test')

In [9]:
train_oof = pd.read_pickle('features/nn_per_feat_oof_raw_train')
test_oof = pd.read_pickle('features/nn_per_feat_oof_raw_test')

In [14]:
from sklearn.preprocessing import MinMaxScaler
#from clr_callback import CyclicLR

model = build_model()
train_oof = np.zeros((train_df.shape[0], len(feature_cols)), dtype=np.float32)
test_oof = np.zeros((test_df.shape[0], len(feature_cols)), dtype=np.float32)

for f_i, f in enumerate(feature_cols):
    fold = 0  
    
    feats = [f, f+'_freq']
    for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
        fold += 1
        
        file_path = "model_weights/{}_fold_{}_feat_{}.hdf5".format(model_prefix, fold, f)
        model.load_weights(file_path)
        
        val = special_reshape(train_df[feats].values[val_ix,:])
        val_y = target[val_ix]
        val_pred = model.predict(val).reshape((len(val_ix),))
        tst_pred = model.predict(special_reshape(test_df[feats].values)).reshape((test_df.shape[0],))
        
        scaler = MinMaxScaler()
        val_pred = scaler.fit_transform(val_pred.reshape((-1, 1))).reshape((-1,))
        tst_pred = scaler.transform(tst_pred.reshape((-1, 1))).reshape((-1,))
        
        train_oof[val_ix, f_i] = val_pred
        test_oof[:, f_i] += tst_pred/fold_num
        
        print("Feat = {}, fold = {}, roc_auc = {:.4f}".format(f, fold, roc_auc_score(val_y, val_pred)))
        
    print('{} overall auc = {:.4f}'.format(f, roc_auc_score(target, train_oof[:, f_i])))
    
K.clear_session()

Feat = var_0, fold = 1, roc_auc = 0.5494
Feat = var_0, fold = 2, roc_auc = 0.5442
Feat = var_0, fold = 3, roc_auc = 0.5477
Feat = var_0, fold = 4, roc_auc = 0.5479
var_0 overall auc = 0.5438
Feat = var_1, fold = 1, roc_auc = 0.5383
Feat = var_1, fold = 2, roc_auc = 0.5491
Feat = var_1, fold = 3, roc_auc = 0.5510
Feat = var_1, fold = 4, roc_auc = 0.5392
var_1 overall auc = 0.5419
Feat = var_2, fold = 1, roc_auc = 0.5510
Feat = var_2, fold = 2, roc_auc = 0.5532
Feat = var_2, fold = 3, roc_auc = 0.5448
Feat = var_2, fold = 4, roc_auc = 0.5499
var_2 overall auc = 0.5463
Feat = var_3, fold = 1, roc_auc = 0.5015
Feat = var_3, fold = 2, roc_auc = 0.5050
Feat = var_3, fold = 3, roc_auc = 0.5051
Feat = var_3, fold = 4, roc_auc = 0.5067
var_3 overall auc = 0.5040
Feat = var_4, fold = 1, roc_auc = 0.5126
Feat = var_4, fold = 2, roc_auc = 0.5038
Feat = var_4, fold = 3, roc_auc = 0.5138
Feat = var_4, fold = 4, roc_auc = 0.5033
var_4 overall auc = 0.5063
Feat = var_5, fold = 1, roc_auc = 0.5259
Feat

Feat = var_42, fold = 1, roc_auc = 0.5081
Feat = var_42, fold = 2, roc_auc = 0.5039
Feat = var_42, fold = 3, roc_auc = 0.5014
Feat = var_42, fold = 4, roc_auc = 0.5028
var_42 overall auc = 0.5050
Feat = var_43, fold = 1, roc_auc = 0.5199
Feat = var_43, fold = 2, roc_auc = 0.5138
Feat = var_43, fold = 3, roc_auc = 0.5263
Feat = var_43, fold = 4, roc_auc = 0.5267
var_43 overall auc = 0.5201
Feat = var_44, fold = 1, roc_auc = 0.5427
Feat = var_44, fold = 2, roc_auc = 0.5454
Feat = var_44, fold = 3, roc_auc = 0.5467
Feat = var_44, fold = 4, roc_auc = 0.5440
var_44 overall auc = 0.5420
Feat = var_45, fold = 1, roc_auc = 0.5218
Feat = var_45, fold = 2, roc_auc = 0.5140
Feat = var_45, fold = 3, roc_auc = 0.5076
Feat = var_45, fold = 4, roc_auc = 0.5205
var_45 overall auc = 0.5173
Feat = var_46, fold = 1, roc_auc = 0.4988
Feat = var_46, fold = 2, roc_auc = 0.5022
Feat = var_46, fold = 3, roc_auc = 0.4995
Feat = var_46, fold = 4, roc_auc = 0.4988
var_46 overall auc = 0.4980
Feat = var_47, fold 

Feat = var_83, fold = 4, roc_auc = 0.5162
var_83 overall auc = 0.5162
Feat = var_84, fold = 1, roc_auc = 0.5020
Feat = var_84, fold = 2, roc_auc = 0.5094
Feat = var_84, fold = 3, roc_auc = 0.5113
Feat = var_84, fold = 4, roc_auc = 0.5135
var_84 overall auc = 0.5092
Feat = var_85, fold = 1, roc_auc = 0.5220
Feat = var_85, fold = 2, roc_auc = 0.5249
Feat = var_85, fold = 3, roc_auc = 0.5229
Feat = var_85, fold = 4, roc_auc = 0.5163
var_85 overall auc = 0.5188
Feat = var_86, fold = 1, roc_auc = 0.5371
Feat = var_86, fold = 2, roc_auc = 0.5406
Feat = var_86, fold = 3, roc_auc = 0.5431
Feat = var_86, fold = 4, roc_auc = 0.5387
var_86 overall auc = 0.5353
Feat = var_87, fold = 1, roc_auc = 0.5290
Feat = var_87, fold = 2, roc_auc = 0.5328
Feat = var_87, fold = 3, roc_auc = 0.5423
Feat = var_87, fold = 4, roc_auc = 0.5353
var_87 overall auc = 0.5334
Feat = var_88, fold = 1, roc_auc = 0.5157
Feat = var_88, fold = 2, roc_auc = 0.5147
Feat = var_88, fold = 3, roc_auc = 0.5176
Feat = var_88, fold 

Feat = var_124, fold = 4, roc_auc = 0.4977
var_124 overall auc = 0.4973
Feat = var_125, fold = 1, roc_auc = 0.5186
Feat = var_125, fold = 2, roc_auc = 0.5170
Feat = var_125, fold = 3, roc_auc = 0.5177
Feat = var_125, fold = 4, roc_auc = 0.5220
var_125 overall auc = 0.5180
Feat = var_126, fold = 1, roc_auc = 0.5074
Feat = var_126, fold = 2, roc_auc = 0.4993
Feat = var_126, fold = 3, roc_auc = 0.5029
Feat = var_126, fold = 4, roc_auc = 0.5021
var_126 overall auc = 0.5032
Feat = var_127, fold = 1, roc_auc = 0.5317
Feat = var_127, fold = 2, roc_auc = 0.5386
Feat = var_127, fold = 3, roc_auc = 0.5370
Feat = var_127, fold = 4, roc_auc = 0.5365
var_127 overall auc = 0.5338
Feat = var_128, fold = 1, roc_auc = 0.5273
Feat = var_128, fold = 2, roc_auc = 0.5266
Feat = var_128, fold = 3, roc_auc = 0.5187
Feat = var_128, fold = 4, roc_auc = 0.5193
var_128 overall auc = 0.5223
Feat = var_129, fold = 1, roc_auc = 0.5059
Feat = var_129, fold = 2, roc_auc = 0.5076
Feat = var_129, fold = 3, roc_auc = 0.

Feat = var_165, fold = 2, roc_auc = 0.5554
Feat = var_165, fold = 3, roc_auc = 0.5512
Feat = var_165, fold = 4, roc_auc = 0.5471
var_165 overall auc = 0.5446
Feat = var_166, fold = 1, roc_auc = 0.5429
Feat = var_166, fold = 2, roc_auc = 0.5487
Feat = var_166, fold = 3, roc_auc = 0.5520
Feat = var_166, fold = 4, roc_auc = 0.5404
var_166 overall auc = 0.5442
Feat = var_167, fold = 1, roc_auc = 0.5297
Feat = var_167, fold = 2, roc_auc = 0.5305
Feat = var_167, fold = 3, roc_auc = 0.5259
Feat = var_167, fold = 4, roc_auc = 0.5295
var_167 overall auc = 0.5243
Feat = var_168, fold = 1, roc_auc = 0.5138
Feat = var_168, fold = 2, roc_auc = 0.5104
Feat = var_168, fold = 3, roc_auc = 0.5047
Feat = var_168, fold = 4, roc_auc = 0.5068
var_168 overall auc = 0.5084
Feat = var_169, fold = 1, roc_auc = 0.5416
Feat = var_169, fold = 2, roc_auc = 0.5377
Feat = var_169, fold = 3, roc_auc = 0.5423
Feat = var_169, fold = 4, roc_auc = 0.5353
var_169 overall auc = 0.5372
Feat = var_170, fold = 1, roc_auc = 0.

In [15]:
train_oof_mean = train_oof.mean(axis=1)
print('Overall auc:', roc_auc_score(target, train_oof_mean))

Overall auc: 0.8139664236972114


In [16]:
fold = 0
for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
    fold += 1

    val_y = target[val_ix]
    val_pred = train_oof_mean[val_ix]
    
    print("Fold = {}, roc_auc = {:.4f}".format(fold, roc_auc_score(val_y, val_pred)))

Fold = 1, roc_auc = 0.8637
Fold = 2, roc_auc = 0.8626
Fold = 3, roc_auc = 0.8695
Fold = 4, roc_auc = 0.8679


In [ ]:
new_feats = [f+'_nn_oof' for f in feature_cols]
pd.DataFrame(data=train_oof, columns=new_feats).to_pickle('features/nn_per_feat_oof_scaled_train')
pd.DataFrame(data=test_oof, columns=new_feats).to_pickle('features/nn_per_feat_oof_scaled_test')
train_oof = pd.read_pickle('features/nn_per_feat_oof_raw_train')
test_oof = pd.read_pickle('features/nn_per_feat_oof_raw_test')